In [3]:
import sys
sys.path.append('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch')
import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from essentials import complete_preprocessing_pipeline
from feature_sets_center_less import GenerateFeatures 
import copy

In [4]:
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_dict_imu.pkl', 'rb') as f:
    imu_dict = pickle.load(f)
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_minze_dict.pkl', 'rb') as f:
    ground_truth_dict = pickle.load(f)
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_dict_urineestimate_method1.pkl', 'rb') as f:
    urine_estimate_dict = pickle.load(f)

Remove the data with the almost the entire data being void.
1. subj_9_void4
2. subj_11_void2

In [5]:
del imu_dict['subj_9_void4']
del imu_dict['subj_11_void2']

Extract windowed features from the 3 axis of the accelerometer, and the 3 axis of the gyroscope.
1. mean
2. rms
3. std
4. range
5. min
6. max
7. time energy
8. permutation entropy
9. spectral entropy
10. spectral_centroid
11. spectral_spread
12. spectral_flatness
13. peak_frequency_ratio
14. hjorth_frequency
15. spectral_skewness
16. spectral_kurtosis
17. snr

In [6]:
data_dict = copy.deepcopy(imu_dict)
labelled_imu_dict = complete_preprocessing_pipeline(data_dict, ground_truth_dict, 
                                target_fs=60,normalize_data=True, use_three_classes=True)

Step 1: Resampling data to 60 Hz...


Step 2: Processing each instance: 100%|██████████| 41/41 [00:00<00:00, 343.34it/s]


In [ ]:
window_sizes = [1,2,3,4,5]
overlaps = [0.0, 0.5, 0.8]

## Three class data
classes: pre-void, void, non-void

In [ ]:
for window_size in window_sizes:
    for overlap in overlaps:
        all_features = []
        for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):
            df = labelled_imu_dict[imu_data]

            analyzer = GenerateFeatures(fs=60, window_duration=window_size, overlap=overlap)
            features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])
            
            table = analyzer.create_summary_table()
            table['experiment_id'] = exp_id + 1
            all_features.append(table)
            
        if overlap == 0.0:
            overlap = 'no'            
        final_features = pd.concat(all_features, ignore_index=True)
        print(f'Window size and overlap: {window_size}s_{overlap}')
        final_features.to_csv(f'/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/pipeline/modified/data_extracted_features/three_class_pp_{window_size}s_{overlap}.csv', index=False)
        final_features.head()

In [7]:
labelled_imu_dict

{'subj_1_void1':            time     acc_x     acc_y     acc_z     gyr_x     gyr_y     gyr_z  \
 0      0.000000  1.304662  0.113255  0.163178 -2.232217  1.042811  0.914265   
 1      0.016672  1.044780 -0.338032  0.446057 -1.553161  0.609596  0.888309   
 2      0.033344  0.670807 -1.234515  0.645578 -0.792932  0.235303  1.014355   
 3      0.050016  0.459069 -1.905110  0.740718 -0.318335  0.055018  1.496646   
 4      0.066688  0.721245 -0.214938  0.909049  0.148847  0.286754  2.206119   
 ...         ...       ...       ...       ...       ...       ...       ...   
 2890  48.182128 -3.338213 -0.750875 -0.009730 -1.577960 -0.359363  0.151838   
 2891  48.198800 -3.127755 -0.598553  0.132317 -1.411974  0.452353  0.342062   
 2892  48.215472 -2.896939  0.135769  0.202553 -0.940317  1.214708  0.525331   
 2893  48.232144 -2.838885  0.350198  0.111096 -0.812362  1.332253  0.600103   
 2894  48.248816 -2.943543 -0.268201 -0.058507 -1.211598  0.974026  0.716968   
 
           label  
 0 

## Two class
classes: non-void, void


In [ ]:
data_dict = copy.deepcopy(imu_dict)
labelled_imu_dict = complete_preprocessing_pipeline(data_dict, ground_truth_dict, 
                                target_fs=60,normalize_data=True, use_three_classes=False)

In [ ]:
window_sizes = [1,2,3,4,5]
overlaps = [0.0, 0.5, 0.8]

In [ ]:
for window_size in window_sizes:
    for overlap in overlaps:
        all_features = []
        for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):
            df = labelled_imu_dict[imu_data]

            analyzer = GenerateFeatures(fs=60, window_duration=window_size, overlap=overlap)
            features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])
            
            table = analyzer.create_summary_table()
            table['experiment_id'] = exp_id + 1
            all_features.append(table)
            
        if overlap == 0.0:
            overlap = 'no'            
        final_features = pd.concat(all_features, ignore_index=True)
        print(f'Window size and overlap: {window_size}s_{overlap}')
        final_features.to_csv(f'/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/pipeline/modified/data_extracted_features/two_class_pp_{window_size}s_{overlap}.csv', index=False)
        final_features.head()